In [13]:
# Unicorn Investment Screening Analysis
##Objective
#To identify high-potential unicorn companies for investment based on:
#- Lower investor count
#- Faster time to unicorn
#- Pre-exit financial stages

#The analysis supports management decision-making through data-driven insights.
import numpy as np
import pandas as pd
import datetime as dt



In [14]:
#Load dataset
df = pd.read_csv("Unicorn_Companies_converted.csv")
df.head()

,Company,Valuation ($B),Date Joined,Country,City,Industry,Select Inverstors,Founded Year,Total Raised,Financial Stage,Investors Count,Deal Terms,Portfolio Exits
0,Bytedance,$140,04-07-2017,China,Beijing,Artificial intelligence,"Sequoia Capital China, SIG Asia Investments, S...",2012,$7.44B,IPO,28.0,8.0,5.0
1,SpaceX,$100.3,12-01-2012,United States,Hawthorne,Other,"Founders Fund, Draper Fisher Jurvetson, Rothen...",2002,$6.874B,NaN,29.0,12.0,NaN
2,Stripe,$95,23-01-2014,United States,San Francisco,Fintech,"Khosla Ventures, LowercaseCapital, capitalG",2010,$2.901B,Asset,39.0,12.0,1.0
3,Klarna,$45.6,12-12-2011,Sweden,Stockholm,Fintech,"Institutional Venture Partners, Sequoia Capita...",2005,$3.472B,Acquired,56.0,13.0,1.0
4,Epic Games,$42,26-10-2018,United States,Cary,Other,"Tencent Holdings, KKR, Smash Ventures",1991,$4.377B,Acquired,25.0,5.0,2.0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1037 entries, 0 to 1036
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Company            1037 non-null   object 
 1   Valuation ($B)     1037 non-null   object 
 2   Date Joined        1037 non-null   object 
 3   Country            1037 non-null   object 
 4   City               1037 non-null   object 
 5   Industry           1037 non-null   object 
 6   Select Inverstors  1036 non-null   object 
 7   Founded Year       1037 non-null   int64  
 8   Total Raised       1013 non-null   object 
 9   Financial Stage    49 non-null     object 
 10  Investors Count    1036 non-null   float64
 11  Deal Terms         1008 non-null   float64
 12  Portfolio Exits    49 non-null     float64
dtypes: float64(3), int64(1), object(9)
memory usage: 68.9+ KB


In [16]:
df.describe()

,Founded Year,Investors Count,Deal Terms,Portfolio Exits
count,1037.000000,1036.000000,1008.000000,49.000000
mean,2012.696239,14.432432,3.093254,1.346939
std,6.022586,9.946537,2.159764,0.751416
min,1919.000000,1.000000,1.000000,1.000000
25%,2011.000000,8.000000,1.000000,1.000000
50%,2014.000000,13.000000,3.000000,1.000000
75%,2016.000000,19.000000,4.000000,1.000000
max,2021.000000,91.000000,19.000000,5.000000


In [17]:
## Data Quality Issue & Resolution

# Some records contained column misalignment due to comma-separated text fields,
# causing geographic values (e.g., Hong Kong, Singapore) to shift across columns.

# The issue was resolved by:
# - Reloading data using Excel-safe formats
# - Applying targeted realignment logic for affected rows
# - Validating geographic and industry consistency post-fix

df = df.rename(columns={'Select Inverstors' : 'Select Investors'})
df.head()

,Company,Valuation ($B),Date Joined,Country,City,Industry,Select Investors,Founded Year,Total Raised,Financial Stage,Investors Count,Deal Terms,Portfolio Exits
0,Bytedance,$140,04-07-2017,China,Beijing,Artificial intelligence,"Sequoia Capital China, SIG Asia Investments, S...",2012,$7.44B,IPO,28.0,8.0,5.0
1,SpaceX,$100.3,12-01-2012,United States,Hawthorne,Other,"Founders Fund, Draper Fisher Jurvetson, Rothen...",2002,$6.874B,NaN,29.0,12.0,NaN
2,Stripe,$95,23-01-2014,United States,San Francisco,Fintech,"Khosla Ventures, LowercaseCapital, capitalG",2010,$2.901B,Asset,39.0,12.0,1.0
3,Klarna,$45.6,12-12-2011,Sweden,Stockholm,Fintech,"Institutional Venture Partners, Sequoia Capita...",2005,$3.472B,Acquired,56.0,13.0,1.0
4,Epic Games,$42,26-10-2018,United States,Cary,Other,"Tencent Holdings, KKR, Smash Ventures",1991,$4.377B,Acquired,25.0,5.0,2.0


In [18]:
#Format to standard column format
df.columns = (
    df.columns
    .str.lower()
    .str.replace(" ", "_")
)
df.columns

Index(['company', 'valuation_($b)', 'date_joined', 'country', 'city',
       'industry', 'select_investors', 'founded_year', 'total_raised',
       'financial_stage', 'investors_count', 'deal_terms', 'portfolio_exits'],
      dtype='object')

In [19]:
#Noticed there were two formats for date_joined: %d-%m-%Y and %m/%d/%Y
#Converting to same format
df['date_joined'] = df['date_joined'].astype(str).str.strip()
df['date_joined_dt'] = pd.NaT
mask_dmy = df['date_joined'].str.fullmatch(r'\d{2}-\d{2}-\d{4}')

df.loc[mask_dmy, 'date_joined_dt'] = pd.to_datetime(
    df.loc[mask_dmy, 'date_joined'],
    format='%d-%m-%Y'
)
mask_mdy = df['date_joined'].str.fullmatch(r'\d{2}/\d{2}/\d{4}')

df.loc[mask_mdy, 'date_joined_dt'] = pd.to_datetime(
    df.loc[mask_mdy, 'date_joined'],
    format='%m/%d/%Y'
)
df['date_joined'] = df['date_joined_dt']
df.drop(columns='date_joined_dt', inplace=True)
df['date_joined'].isna().sum()
df['date_joined']

0      2017-07-04
1      2012-01-12
2      2014-01-23
3      2011-12-12
4      2018-10-26
          ...    
1032   2022-02-22
1033   2022-02-23
1034   2022-02-23
1035   2022-02-23
1036   2022-02-24
Name: date_joined, Length: 1037, dtype: datetime64[ns]

In [20]:
#Make a new Year Column
df['year'] = pd.to_datetime(df['date_joined']).dt.year
df.head()

,company,valuation_($b),date_joined,country,city,industry,select_investors,founded_year,total_raised,financial_stage,investors_count,deal_terms,portfolio_exits,year
0,Bytedance,$140,2017-07-04,China,Beijing,Artificial intelligence,"Sequoia Capital China, SIG Asia Investments, S...",2012,$7.44B,IPO,28.0,8.0,5.0,2017
1,SpaceX,$100.3,2012-01-12,United States,Hawthorne,Other,"Founders Fund, Draper Fisher Jurvetson, Rothen...",2002,$6.874B,NaN,29.0,12.0,NaN,2012
2,Stripe,$95,2014-01-23,United States,San Francisco,Fintech,"Khosla Ventures, LowercaseCapital, capitalG",2010,$2.901B,Asset,39.0,12.0,1.0,2014
3,Klarna,$45.6,2011-12-12,Sweden,Stockholm,Fintech,"Institutional Venture Partners, Sequoia Capita...",2005,$3.472B,Acquired,56.0,13.0,1.0,2011
4,Epic Games,$42,2018-10-26,United States,Cary,Other,"Tencent Holdings, KKR, Smash Ventures",1991,$4.377B,Acquired,25.0,5.0,2.0,2018


In [21]:
#Total years to become a unicorn
df['years_to_join'] = df['year'] - df['founded_year']
df.head()

,company,valuation_($b),date_joined,country,city,industry,select_investors,founded_year,total_raised,financial_stage,investors_count,deal_terms,portfolio_exits,year,years_to_join
0,Bytedance,$140,2017-07-04,China,Beijing,Artificial intelligence,"Sequoia Capital China, SIG Asia Investments, S...",2012,$7.44B,IPO,28.0,8.0,5.0,2017,5
1,SpaceX,$100.3,2012-01-12,United States,Hawthorne,Other,"Founders Fund, Draper Fisher Jurvetson, Rothen...",2002,$6.874B,NaN,29.0,12.0,NaN,2012,10
2,Stripe,$95,2014-01-23,United States,San Francisco,Fintech,"Khosla Ventures, LowercaseCapital, capitalG",2010,$2.901B,Asset,39.0,12.0,1.0,2014,4
3,Klarna,$45.6,2011-12-12,Sweden,Stockholm,Fintech,"Institutional Venture Partners, Sequoia Capita...",2005,$3.472B,Acquired,56.0,13.0,1.0,2011,6
4,Epic Games,$42,2018-10-26,United States,Cary,Other,"Tencent Holdings, KKR, Smash Ventures",1991,$4.377B,Acquired,25.0,5.0,2.0,2018,27


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1037 entries, 0 to 1036
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   company           1037 non-null   object        
 1   valuation_($b)    1037 non-null   object        
 2   date_joined       1037 non-null   datetime64[ns]
 3   country           1037 non-null   object        
 4   city              1037 non-null   object        
 5   industry          1037 non-null   object        
 6   select_investors  1036 non-null   object        
 7   founded_year      1037 non-null   int64         
 8   total_raised      1013 non-null   object        
 9   financial_stage   49 non-null     object        
 10  investors_count   1036 non-null   float64       
 11  deal_terms        1008 non-null   float64       
 12  portfolio_exits   49 non-null     float64       
 13  year              1037 non-null   int32         
 14  years_to_join     1037 n

In [23]:
#Drop rows which have incomplete data with which the findings will get skewed
df = df.dropna(subset= [
               'total_raised',
               'investors_count'
])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1012 entries, 0 to 1036
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   company           1012 non-null   object        
 1   valuation_($b)    1012 non-null   object        
 2   date_joined       1012 non-null   datetime64[ns]
 3   country           1012 non-null   object        
 4   city              1012 non-null   object        
 5   industry          1012 non-null   object        
 6   select_investors  1011 non-null   object        
 7   founded_year      1012 non-null   int64         
 8   total_raised      1012 non-null   object        
 9   financial_stage   48 non-null     object        
 10  investors_count   1012 non-null   float64       
 11  deal_terms        994 non-null    float64       
 12  portfolio_exits   48 non-null     float64       
 13  year              1012 non-null   int32         
 14  years_to_join     1012 non-nu

In [26]:
#Find how many companies are actually eligible for VC investing
excluded_stages = ['IPO', 'Acquired', 'Divestiture']
df['investment_eligible'] = ~df['financial_stage'].isin(excluded_stages)
df['investment_eligible'].value_counts()

investment_eligible
True     970
False     42
Name: count, dtype: int64

In [34]:
#We have shortlisted 970 companies
#Filter to eligible companies
eligible_df = df[df['investment_eligible']]
eligible_df.head()


,company,valuation_($b),date_joined,country,city,industry,select_investors,founded_year,total_raised,financial_stage,investors_count,deal_terms,portfolio_exits,year,years_to_join,investment_eligible
1,SpaceX,$100.3,2012-01-12,United States,Hawthorne,Other,"Founders Fund, Draper Fisher Jurvetson, Rothen...",2002,$6.874B,NaN,29.0,12.0,NaN,2012,10,True
2,Stripe,$95,2014-01-23,United States,San Francisco,Fintech,"Khosla Ventures, LowercaseCapital, capitalG",2010,$2.901B,Asset,39.0,12.0,1.0,2014,4,True
5,Canva,$40,2018-08-01,Australia,Surry Hills,Internet software & services,"Sequoia Capital China, Blackbird Ventures, Mat...",2012,$571.26M,NaN,26.0,8.0,NaN,2018,6,True
6,Checkout.com,$40,2019-02-05,United Kingdom,London,Fintech,"Tiger Global Management, Insight Partners, DST...",2012,$1.83B,NaN,15.0,4.0,NaN,2019,7,True
7,Instacart,$39,2014-12-30,United States,San Francisco,"Supply chain, logistics, & delivery","Khosla Ventures, Kleiner Perkins Caufield & By...",2012,$2.686B,NaN,29.0,12.0,NaN,2014,2,True


In [36]:
#Now, we need to define scoring parameter to fairly compare the companies on two parameters defined by management:
# 1. The number of investors should be less
# 2. The time it took to reach Unicorn level should be less
#First we define a function to normalize metrics

def max_min_normalize(x):
    return (x - x.min())/(x.max() - x.min())

#Normalize number of investors
eligible_df['investor_score'] = 1 - max_min_normalize(eligible_df['investors_count'])

#Normalize time taken to reach unicorn level
eligible_df['time_score'] = 1 - max_min_normalize(eligible_df['years_to_join'])

#Compile them into an investment score
eligible_df['investment_score'] = (0.5*eligible_df['investor_score'] + 0.5*eligible_df['time_score'])

#Final DF
final_cols = [
    'company',
    'valuation_($b)',
    'country',
    'city',
    'industry',
    'total_raised',
    'founded_year',
    'deal_terms',
    'portfolio_exits',
    'select_investors',
    'investor_score',
    'time_score',
    'investment_score'
]

final_df = eligible_df[final_cols]
final_df.info()

<ipython-input-36-e697a0ce538f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eligible_df['investor_score'] = 1 - max_min_normalize(eligible_df['investors_count'])
<ipython-input-36-e697a0ce538f>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eligible_df['time_score'] = 1 - max_min_normalize(eligible_df['years_to_join'])
<ipython-input-36-e697a0ce538f>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

<class 'pandas.core.frame.DataFrame'>
Index: 970 entries, 1 to 1036
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   company           970 non-null    object 
 1   valuation_($b)    970 non-null    object 
 2   country           970 non-null    object 
 3   city              970 non-null    object 
 4   industry          970 non-null    object 
 5   total_raised      970 non-null    object 
 6   founded_year      970 non-null    int64  
 7   deal_terms        953 non-null    float64
 8   portfolio_exits   6 non-null      float64
 9   select_investors  969 non-null    object 
 10  investor_score    970 non-null    float64
 11  time_score        970 non-null    float64
 12  investment_score  970 non-null    float64
dtypes: float64(5), int64(1), object(7)
memory usage: 79.6+ KB


In [42]:

final_df.to_csv(
    "processed_unicorn_data.csv",
    index=False
)
